In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [2]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = '20180713_wp250rev_01_en_C521EC14-99FD-FB6A-F550FB846F60A5C1_49827.pdf'
paper_name = "Guidelines on Personal data breach notification under Regulation 2016/679"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

In [112]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 4, end_page = 0, header_size=70, footer_size=90, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

file = "../tmp/data_breach.md"
with open(file, 'w', encoding='utf-8') as f:
    f.write(document_text)

TypeError: object of type 'DataBreach' has no len()

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/data_breach.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'\b(I|II|III|IV|V|VI|VII)\.\s*(.*)')
subsection_pattern = re.compile(r'([A-Z])\.\s*(.*)') 
point_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = section + "." + subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = section + "." + subsection + "." + point
            data.append([section, subsection, point, heading, text, section_reference])
        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)

df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[0:20]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
0,,,,False,INTRODUCTION,INTRODUCTION
1,,,,False,The General Data Protection Regulation (the GD...,INTRODUCTION
2,,,,False,Obligations to notify in cases of breaches pre...,INTRODUCTION
3,,,,False,The Article 29 Working Party (WP29) considers ...,INTRODUCTION
4,,,,False,Controllers and processors are therefore encou...,INTRODUCTION
5,,,,False,The GDPR contains provisions on when a breach ...,INTRODUCTION
6,,,,False,In its Opinion 03/2014 on personal data breach...,INTRODUCTION
7,,,,False,The current Guidelines explain the mandatory b...,INTRODUCTION
8,I,,,True,Personal data breach notification under the GDPR,I
9,I,A,,True,Basic security considerations,I.A


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 2 augmented with footnotes
Row 3 augmented with footnotes
Row 4 augmented with footnotes
Row 6 augmented with footnotes
Row 10 augmented with footnotes
Row 11 augmented with footnotes
Row 20 augmented with footnotes
Row 23 augmented with footnotes
Row 26 augmented with footnotes
Row 33 augmented with footnotes
Row 40 augmented with footnotes
Row 41 augmented with footnotes
Row 45 augmented with footnotes
Row 51 augmented with footnotes
Row 55 augmented with footnotes
Row 63 augmented with footnotes
Row 67 augmented with footnotes
Row 75 augmented with footnotes
Row 78 augmented with footnotes
Row 106 augmented with footnotes
Row 118 augmented with footnotes
Row 124 augmented with footnotes
Row 126 augmented with footnotes
Row 130 augmented with footnotes
Row 132 augmented with footnotes
Row 135 augmented with footnotes
Row 148 augmented with footnotes
Row 174 augmented with footnotes
Row 175 augmented with footnotes
Row 183 augmented with footnotes
Ro

In [6]:
file = "../../inputs/documents/data_breach.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [11]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.data_breach
importlib.reload(gdpr_rag.documents.data_breach)
from gdpr_rag.documents.data_breach import DataBreach

path_to_manual_as_csv_file = "../../inputs/documents/data_breach.parquet"

doc = DataBreach(path_to_manual_as_csv_file)


In [12]:
from IPython.display import Markdown, display

section = "I.A"
section = "II.B.2"
section = "VII.B"

section = "INTRODUCTION"
section = "I.B.3"
# section = "2.1.3"
# section = "3.3"

# section = "1.1"


print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


I Personal data breach notification under the GDPR
I.B What is a personal data breach?
I.B.3 The possible consequences of a personal data breach


# I Personal data breach notification under the GDPR

## I.B What is a personal data breach?

### I.B.3 The possible consequences of a personal data breach

A breach can potentially have a range of significant adverse effects on individuals, which can result in physical, material, or non-material damage. The GDPR explains that this can include loss of control over their personal data, limitation of their rights, discrimination, identity theft or fraud, financial loss, unauthorised reversal of pseudonymisation, damage to reputation, and loss of confidentiality of personal data protected by professional secrecy. It can also include any other significant economic or social disadvantage to those individuals[^17].

Accordingly, the GDPR requires the controller to notify a breach to the competent supervisory authority, unless it is unlikely to result in a risk of such adverse effects taking place. Where there is a likely high risk of these adverse effects occurring, the GDPR requires the controller to communicate the breach to the affected individuals as soon as is reasonably feasible[^18].

The importance of being able to identify a breach, to assess the risk to individuals, and then notify if required, is emphasised in Recital 87 of the GDPR:

"It should be ascertained whether all appropriate technological protection and organisational measures have been implemented to establish immediately whether a personal data breach has taken place and to inform promptly the supervisory authority and the data subject. The fact that the notification was made without undue delay should be established taking into account in particular the nature and gravity of the personal data breach and its consequences and adverse effects for the data subject. Such notification may result in an intervention of the supervisory authority in accordance with its tasks and powers laid down in this Regulation."

Further guidelines on assessing the risk of adverse effects to individuals are considered in section IV.

If controllers fail to notify either the supervisory authority or data subjects of a data breach or both even though the requirements of Articles 33 and/or 34 are fulfilled, then the supervisory authority is presented with a choice that must include consideration of all of the corrective measures at its disposal, which would include consideration of the imposition of the appropriate administrative fine[^19], either accompanying a corrective measure under Article 58(2) or on its own. Where an administrative fine is chosen, its value can be up to 10,000,000 EUR or up to 2 % if the total worldwide annual turnover of an undertaking under Article 83(4)(a) of the GDPR. It is also important to bear in mind that in some cases, the failure to notify a breach could reveal either an absence of existing security measures or an inadequacy of the existing security measures. The WP29 guidelines on administrative fines state: "The occurrence of several different infringements committed together in any particular single case means that the supervisory authority is able to apply the administrative fines at a level which is effective, proportionate and dissuasive within the limit of the gravest infringement". In that case, the supervisory authority will also have the possibility to issue sanctions for failure to notify or communicate the breach (Articles 33 and 34) on the one hand, and absence of (adequate) security measures (Article 32) on the other hand, as they are two separate infringements.

  
[^17]: See also Recitals 85 and 75  
[^18]: See also Recital 86.  
[^19]: For further details, please see WP29 Guidelines on the application and setting of administrative fines, available here: http://ec.europa.eu/newsroom/just/document.cfm?doc_id=47889

In [75]:
rc = doc.reference_checker
rc.is_valid("VII.B")

True